In [ ]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne

import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine

# TODO: hmm fix these utils imports, import the funcs individually. 6/1/25.
from src.analysis.utils.general_utils import *
from src.analysis.utils.general_utils import make_or_load_subjects_electrodes_to_ROIs_dict
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup

from src.analysis.config import experiment_conditions

from src.analysis.utils.labeled_array_utils import (
    put_data_in_labeled_array_per_roi_subject,
    remove_nans_from_labeled_array,
    remove_nans_from_all_roi_labeled_arrays,
    concatenate_conditions_by_string,
    get_data_in_time_range
)

from src.analysis.decoding.decoding import (
    process_and_balance_data_for_decoding, 
    get_and_plot_confusion_matrix_for_rois_jim,
    Decoder, 
    windower,
    get_confusion_matrices_for_rois_time_window_decoding_jim,
    compute_accuracies,
    perform_time_perm_cluster_test_for_accuracies,
    plot_accuracies
)

import mne.time_frequency
from ieeg.calc.scaling import rescale
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np
from src.analysis.spec.wavelet_functions import get_uncorrected_wavelets, get_uncorrected_multitaper

#### 0. Load data

In [ ]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

# let's make a dictionary where the keys are output names and the values are lists of corresponding events
output_names_and_events_dict = {}
# output_names_and_events_dict['Stimulus_c25and75_fixationCrossBase_0.5sec'] = ["Stimulus/c75.0", "Stimulus/c25.0"]
# output_names_and_events_dict['Stimulus_i25and75_fixationCrossBase_0.5sec'] = ["Stimulus/i75.0", "Stimulus/i25.0"]
# output_names_and_events_dict['Stimulus_s25and75_fixationCrossBase_0.5sec'] = ['Stimulus/i75.0/s25.0', 'Stimulus/i25.0/s75.0', 'Stimulus/c75.0/s25.0', 'Stimulus/c25.0/s75.0']
# output_names_and_events_dict['Stimulus_r25and75_fixationCrossBase_0.5sec'] = ['Stimulus/i75.0/r25.0', 'Stimulus/i25.0/r75.0', 'Stimulus/c75.0/r25.0', 'Stimulus/c25.0/r75.0']
output_names_and_events_dict['Stimulus_fixationCrossBase_0.5sec'] = ["Stimulus"]

#### 1. For each electrode, make multitaper using all training trials, for both conditions to be compared

In [ ]:
# TODO: Turn this cell into a function

baseline_times = [-0.5, 0]
signal_times = [-0.5, 1.5]

# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']

subjects = ['D0057']

subjects_tfr_objects = {}

# TODO: Kind of make a subjects_mne_objects thing but for wavelets/multitaper i think. Need to store data across subjects.
for sub in subjects:
    subjects_tfr_objects[sub] = {}
    
    # load in good data so we can use it for gettin bad channels and getting filenames
    good = get_good_data(sub, layout)

    # make stimulus baseline EpochsTFR
    base = get_uncorrected_multitaper(sub, layout, events=["Stimulus"], times=baseline_times)

    # make signal wavelets
    for output_name, events in output_names_and_events_dict.items():
        spec = get_uncorrected_multitaper(sub, layout, events, signal_times)
        spec_rescaled = rescale(spec, base, copy=True, mode='ratio').average(
            lambda x: np.nanmean(x, axis=0), copy=True)
        spec_rescaled._data = np.log10(spec_rescaled._data) * 20 # convert to dB
        fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
        spec_rescaled.info['subject_info']['files'] = tuple(fnames)
        spec_rescaled.info['bads'] = good.info['bads']

#### 2. Find clusters that are significantly different between the two conditions, in the multitaper spectrogram


In [ ]:
time_perm_cluster(spec_rescaled_condition_1, spec_rescaled_condition_2)

#### 3. Train a decoder on just the significant time-frequency clusters, test on the test trials

#### 4. Repeat but with new test trials (cross-validate)

#### 5. Figure out a way of plotting this. This is univariate approach, make things modular so I can do multivariate later once I figure that out.